In [7]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [8]:
## dropping missing values
heart = heart.dropna()

In [24]:
## defining input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## splitting the data into 80% training and 20% testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [25]:
## scaling the data
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [26]:
## identifying important variables with lasso
lasso_md = LassoCV().fit(X_train, Y_train)

## extracting the best lambda value
lasso_lambda = lasso_md.alpha_

## Fitting the lasso model with best lambda
lasso_md = Lasso(alpha = lasso_lambda).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.06136497,  0.29707372,  0.04740824,  0.00600885,  0.49665455,
        0.        , -0.        ,  0.31112591])

Removing heart rate, BMI, and totChol because they have the lowest

In [30]:
## defining the input and target variables for logistic regression
X_train_logit = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_logit = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

## building logistic regression model
logit_md = LogisticRegression().fit(X_train_logit, Y_train)

## predicting on the test
logit_pred = logit_md.predict_proba(X_test_logit)[:, 1]

## changing likelihoods to labels
logit_label = np.where(logit_pred < 0.1, 0, 1)
logit_label

## Computing the recall
print('The Recall Score is for the Logistic Regression Model is:', recall_score(Y_test, logit_label))

The Recall Score is for the Logistic Regression Model is: 0.8839285714285714


In [32]:
X_train.head()

,male,age,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
0,0.0,0.945946,0.0,0.283368,0.513002,0.798825,0.102041,0.062147
1,0.0,0.648649,0.0,0.149897,0.338061,0.247342,0.367347,0.098870
2,1.0,0.567568,1.0,0.158111,0.271868,0.153050,0.714286,0.129944
3,0.0,0.567568,0.0,0.316222,0.323877,0.362339,0.500000,0.177966
4,1.0,0.270270,0.0,0.236140,0.219858,0.260772,0.408163,0.101695


In [34]:
## Random Forest Model
rf_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## extracting the feature importances
importances = pd.DataFrame({'Feature': X_train.columns, 'Importances': rf_md.feature_importances_})

## printing the importances (sorting from most important to least important)
## higher numbers are the better
importances = importances.sort_values(by = 'Importances', ascending = False)
importances

,Feature,Importances
4,sysBP,0.189406
5,BMI,0.174150
3,totChol,0.159142
1,age,0.155768
7,glucose,0.153906
6,heartRate,0.122604
0,male,0.023175
2,currentSmoker,0.021849


In [36]:
## building the RF model using the top 5 important features
X_train_rf = X_train[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]
X_test_rf = X_test[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]

## building the random forest model
rf_md = RandomForestClassifier(n_estimators = 500).fit(X_train_rf, Y_train)

## predicting on the test
rf_pred = logit_md.predict_proba(X_test_rf)[:, 1]

## changing likelihoods to labels
rf_label = np.where(rf_pred < 0.1, 0, 1)
rf_label

## Computing the recall
print('The Recall Score is for the Random Forest Model is:', recall_score(Y_test, rf_label))

The Recall Score is for the Random Forest Model is: 0.7142857142857143


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- BMI
- totChol
Feature names seen at fit time, yet now missing:
- currentSmoker
- male

  warnings.warn(message, FutureWarning)
